<div class="alert alert-block alert-info">
<b>In3dSlicer GUI</b>:
Load MRI last folder
Load markers templateSave project and files in a new folder inside the individual, make sure to change the default folder for all files.
In markup module, create two point lists: femur_head_right and femur_head_left.

For each, left and right accordingly, collect lms on femur heads circular outline, red and green slices: move slices to the level of the largest femur head radius (approximately near fovea).

Create a new empty markups ladmark list 'lms'.

Once finished, follow instructions below: either or execute code or interact with <b>GUI</b>.
</div>

In [ ]:
import numpy as np
import slicer
import math
import vtk

In [ ]:
# helper functions:

def sphereFit(data):

    #   Assemble the A matrix
    spX = np.array(data[:,0])
    spY = np.array(data[:,1])
    spZ = np.array(data[:,2])
    A = np.zeros((len(spX),4))
    A[:,0] = spX*2
    A[:,1] = spY*2
    A[:,2] = spZ*2
    A[:,3] = 1

    #   Assemble the f matrix
    f = np.zeros((len(spX),1))
    f[:,0] = (spX*spX) + (spY*spY) + (spZ*spZ)
    C, residules, rank, singval = np.linalg.lstsq(A,f, rcond = None)

    #   solve for the radius
    t = np.dot(C[0],C[0])+np.dot(C[1],C[1])+np.dot(C[2],C[2])+C[3]
 
    radius = math.sqrt(t[0])
    center = np.array([C[0], C[1], C[2]])

    return radius, center

def setSlicePoseFromSliceNormalAndPosition(sliceNode, sliceNormal, slicePosition, defaultViewUpDirection=None, backupViewRightDirection=None):
    
    ## Fix up input directions
    if defaultViewUpDirection is None:
        defaultViewUpDirection = [0,0,1]
    if backupViewRightDirection is None:
        backupViewRightDirection = [-1,0,0]
    if sliceNormal[1]>=0:
        sliceNormalStandardized = sliceNormal
    else:
        sliceNormalStandardized = [-sliceNormal[0], -sliceNormal[1], -sliceNormal[2]]

        ## Compute slice axes
    sliceNormalViewUpAngle = vtk.vtkMath.AngleBetweenVectors(sliceNormalStandardized, defaultViewUpDirection)
    angleTooSmallThresholdRad = 0.25 # about 15 degrees
    if sliceNormalViewUpAngle > angleTooSmallThresholdRad and sliceNormalViewUpAngle < vtk.vtkMath.Pi() - angleTooSmallThresholdRad:
        viewUpDirection = defaultViewUpDirection
        sliceAxisY = viewUpDirection
        sliceAxisX = [0, 0, 0]
        vtk.vtkMath.Cross(sliceAxisY, sliceNormalStandardized, sliceAxisX)
    else:
        sliceAxisX = backupViewRightDirection
    
    ## Set slice axes
    sliceNode.SetSliceToRASByNTP(sliceNormalStandardized[0], sliceNormalStandardized[1], sliceNormalStandardized[2],
        sliceAxisX[0], sliceAxisX[1], sliceAxisX[2],
        slicePosition[0], slicePosition[1], slicePosition[2], 0)

In [ ]:
# left femur
left_fh = slicer.util.getNode('femur_head_left')
left_fh = slicer.util.arrayFromMarkupsControlPoints(left_fh)
left_radius, left_center = sphereFit(left_fh)
left_center

In [ ]:
# right femur
right_fh = slicer.util.getNode('femur_head_right')
right_fh = slicer.util.arrayFromMarkupsControlPoints(right_fh)
right_radius, right_center = sphereFit(right_fh)
right_radius

In [ ]:
# Import lms and add center points to it
lms = slicer.util.getNode('lms')
left_center_point = lms.AddControlPoint(left_center)
lms.SetNthControlPointLabel(left_center_point, 'femur_l_center')
right_center_point = lms.AddControlPoint(right_center)
lms.SetNthControlPointLabel(right_center_point, 'femur_r_center')

<div class="alert alert-block alert-info">
    <b>In 3dSlicer GUI</b>, place the following key markers:
    
 1) _ASIS_l_ and _ASIS_r_ : defined as the anteriormost point of the iliac blades that touches skin on the red slice; on the green slice, the same point should look like as small round shadow and on the yellow slice it is anteroinferior point of the iliac blade.
    
 2) _PSIS_l and _PSIS_r_ : <b>CAREFUL! these are essential and an error will propagate through the rest of data collection.</b> defined as posterior and iferior points of iliac blades. They should be positioned posterior to sacrum. On the red slice, the sacrum must still be visible, probably it's uppermost vertebra.
</div>

In [ ]:
# re-load lms node
lms = slicer.util.getNode('lms')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)
asis_l = lms_list[lms.GetControlPointIndexByLabel('ASIS_l')]
asis_r = lms_list[lms.GetControlPointIndexByLabel('ASIS_r')]
pelvis_origin = np.mean((asis_l, asis_r), axis = 0)
pelvis_origin_point = lms.AddControlPoint(vtk.vtkVector3d(pelvis_origin))
lms.SetNthControlPointLabel(pelvis_origin_point, 'pelvis_origin')
psis_l = lms_list[lms.GetControlPointIndexByLabel('PSIS_l')]
psis_r = lms_list[lms.GetControlPointIndexByLabel('PSIS_r')]

In [ ]:
# define new normal for the red slice
red_slice_new_normal = np.cross((pelvis_origin - psis_l), (pelvis_origin - psis_r))
red_slice_new_normal = red_slice_new_normal/np.linalg.norm(red_slice_new_normal)

In [ ]:
red_slice_new_normal

In [ ]:
# reorient red slice
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(
    redSliceNode, red_slice_new_normal, 
    np.mean((psis_l, psis_r, pelvis_origin), 
            axis = 0)) ## red slice now shows PSIS_l, PSIS_r and pelvis_origin points

In [ ]:
# reorient green slice
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(
    greenSliceNode, np.array([0,-1,0]), 
    np.mean((psis_l, psis_r, pelvis_origin), 
            axis = 0)) ## green slice now crossects pelvis in the middle

<div class="alert alert-block alert-info">
<b>In 3dSclier GUI, collect a number of key markers:</b>
    
1)  _ilium_l_ and _ilium_r_ : move the red slice up so that it touches the tops of iliac crests. The green slice position marks the centre between psis and pelvis origin markers. The red slice should show an elongated shadow of the left or/and right iliac crests are visible but do not quite reach the green line, which marks the position of the green slice when observed on the red. The points should be placed at the anterior corner of the right or left iliac crest shadow. Once done, return the red slice to default poisiton (see panel above the slice, change 'reformat' to 'axial'). 

2) _knee_l_lat_ and _knee_r_lat_ : these shoule be placed on the lateralmost points of lateral femoral epicondyles.

3) _ankle_l_lat_ and _ankle_l_lat_ : these should be placed on lateralmost points of fibulas# lateral malleolae.

<b> Create a new lm markup, name it any way you like, and do the following:</b>
    
1) From _orientation_template_, copy points that match those already in lms to the new lm markup; 
2) Change the list position of the points so that your new markup is identical to lms;
3) Use module 'IGT' -> 'fiducial registration' to create new transform from your new markup to lms (warping); 
4) Move the _orientation_template_ landmarks to the transform dependancy to check points fit the image; 
5) If the above answer is 'yes', return _orientation_template_ to the independent position, clone it and rename the clone to 'orientation'.
6) Move the 'orientation' clone to the transform dependentcy and harden the transform.

Superimposed landmarks should fit exactly.
</div>

<div class="alert alert-block alert-info">
<b>Adjust location of projected landmarks, starting from the top:</b>

_gr_troch_as_l_ and _gr_troch_as_r_** -- greater trochanter anterosuperior -- defined as the anterior point on the smallest red crossection through the greater trochanter, roughly corresponds with the superior point of the greater trochanter anatomically; one needs to be careful to distinguish between the bone, fat and tendons

_gr_troch_lat_l_ and _gr_troch_lat_r_** -- greater trochanter lateral -- defined as the lateralmost point on the green slice through greater trochanter, it should lie posteriorly to the gr_troch_as point

**__NB position of trochanter points is likely to be influenced by the rotaion of the femur: watch the effects__

_isch_tuber_l_ and _isch_tuber_r_ -- ischial tuberosity -- defined as the posteriormost landmark on ischial tuberosity, on the green slice is marked as black endpoint of the bone

_pub_infer_c_ -- pubis inferior central -- inferiormost point of the pubic sympysis impriz on the central yellow slice (controlled by the red and green)

_isch_infer_l_ and _isch_infer-r_ -- ischial [bone] inferior -- defined as infero-anteriormost point of the imprint of the ischio-pubic arch on the red slice (control on the green and yellow)

**__Here, one needs to skip midshaft points as their position depends on the axis of the femur and therefore its inclination relative to slices - they will be adjusted later__

_knee_l_med_ and _knee_r_med_ -- knee medial, defined as the medial-most point of femur medial epicondyle on the red slice; this point should be roughly inline with the lateral knee point and the center of the intercondylar fossa

_ankle_l_med_ and _ankle_r_med_ -- medial malleolus on left and right tibias; defined as the most medially prominent point on tibial malleolus
</div>

In [ ]:
# Calculate and update central points for the knee and the ankle
## note that knee needs two sets of points updated: the ones that will be in child and the ones that will be in parent frames

lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_lat = lms_list[lms.GetControlPointIndexByLabel('knee_l_lat')]
knee_l_med = lms_list[lms.GetControlPointIndexByLabel('knee_l_med')]
knee_l_center = lms.GetControlPointIndexByLabel('knee_l_center')
knee_l_center_in_femur_l = lms.GetControlPointIndexByLabel('knee_l_center_in_femur_l')
temp1 = np.mean((knee_l_lat, knee_l_med), axis = 0)
lms.SetNthControlPointPosition(knee_l_center, temp1[0], temp1[1], temp1[2])
lms.SetNthControlPointPosition(knee_l_center_in_femur_l, temp1[0], temp1[1], temp1[2])

knee_r_lat = lms_list[lms.GetControlPointIndexByLabel('knee_r_lat')]
knee_r_med = lms_list[lms.GetControlPointIndexByLabel('knee_r_med')]
knee_r_center = lms.GetControlPointIndexByLabel('knee_r_center')
knee_r_center_in_femur_r = lms.GetControlPointIndexByLabel('knee_r_center_in_femur_r')
temp2 = np.mean((knee_r_lat, knee_r_med), axis = 0)
lms.SetNthControlPointPosition(knee_r_center, temp2[0], temp2[1], temp2[2])
lms.SetNthControlPointPosition(knee_r_center_in_femur_r, temp2[0], temp2[1], temp2[2])

ankle_l_lat = lms_list[lms.GetControlPointIndexByLabel('ankle_l_lat')]
ankle_l_med = lms_list[lms.GetControlPointIndexByLabel('ankle_l_med')]
ankle_l_center = lms.GetControlPointIndexByLabel('ankle_l_center')
temp3 = np.mean((ankle_l_lat, ankle_l_med), axis = 0)
lms.SetNthControlPointPosition(ankle_l_center, temp3[0], temp3[1], temp3[2])

ankle_r_lat = lms_list[lms.GetControlPointIndexByLabel('ankle_r_lat')]
ankle_r_med = lms_list[lms.GetControlPointIndexByLabel('ankle_r_med')]
ankle_r_center = lms.GetControlPointIndexByLabel('ankle_r_center')
temp4 = np.mean((ankle_r_lat, ankle_r_med), axis = 0)
lms.SetNthControlPointPosition(ankle_r_center, temp4[0], temp4[1], temp4[2])

## Locate lat and med points on tibias

In [ ]:
# First, calculate tibial axes
lms = getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]

ankle_l_center = lms_list[lms.GetControlPointIndexByLabel('ankle_l_center')]
ankle_r_center = lms_list[lms.GetControlPointIndexByLabel('ankle_r_center')]

tibia_l_normal = (knee_l_center-ankle_l_center)/np.linalg.norm(knee_l_center-ankle_l_center)
tibia_r_normal = (knee_r_center-ankle_r_center)/np.linalg.norm(knee_r_center-ankle_r_center)b

### Left tibia

In [ ]:
# Second, reorient red slice perpendicular to ankle-knee axis for the left tibia
redSliceNode = getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, knee_l_center)

<div class="alert alert-block alert-info">
    <b>In 3dSclicer GUI</b>, move red slice to intersect with the uppermost part of tibial plateau, place lat med and central points across the longest axis with the central point roughly under the lateral intercondylar tubercle.
</div>

### Right tibia

In [ ]:
# Third, reorient red slice perpendicular to ankle-knee axis for the right tibia

redSliceNode = getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, knee_r_center)


<div class="alert alert-block alert-info">
    <b>In3dSlicer GUI</b>, move red slice to intersect with the uppermost part of tibial plateau, place lat med and central points across the longest axis with the central point roughly under the lateral intercondylar tubercle.
</div>

## Locate midshaft points on tibias, fibulas and femora

In [ ]:
## Left tibia
fibula_l_midshaft_anter = lms.GetControlPointIndexByLabel('fibula_l_midshaft_anter')
tibia_l_midshaft_anter = lms.GetControlPointIndexByLabel('tibia_l_midshaft_anter')
tibia_l_midshaft_poster = lms.GetControlPointIndexByLabel('tibia_l_midshaft_poster')
tibia_l_mean = np.mean((knee_l_center,ankle_l_center), axis=0)

In [ ]:
### the next step will locate midshaft landmarks APPROXIMATELY in the middle of diaphisis
lms.SetNthControlPointPosition(fibula_l_midshaft_anter, 
                               lms_list[fibula_l_midshaft_anter][0], 
                               lms_list[fibula_l_midshaft_anter][1], 
                               tibia_l_mean[2])
lms.SetNthControlPointPosition(tibia_l_midshaft_anter, 
                               lms_list[tibia_l_midshaft_anter][0], 
                               lms_list[tibia_l_midshaft_anter][1], 
                               tibia_l_mean[2])
lms.SetNthControlPointPosition(tibia_l_midshaft_poster, 
                               lms_list[tibia_l_midshaft_poster][0], 
                               lms_list[tibia_l_midshaft_poster][1], 
                               tibia_l_mean[2])

In [ ]:
### reorient the red slice:
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, tibia_l_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, adjust landmarks position to fit on the above slice exactly anterior or posterior of the shaft.
</div>

In [ ]:
## Right tibia
fibula_r_midshaft_anter = lms.GetControlPointIndexByLabel('fibula_r_midshaft_anter')
tibia_r_midshaft_anter = lms.GetControlPointIndexByLabel('tibia_r_midshaft_anter')
tibia_r_midshaft_poster = lms.GetControlPointIndexByLabel('tibia_r_midshaft_poster')
tibia_r_mean = np.mean((knee_r_center,ankle_r_center), axis=0)

In [ ]:
### the next step will locate midshaft landmars APPROXIMATELY in the middle of diaphisis
lms.SetNthControlPointPosition(fibula_r_midshaft_anter, 
                               lms_list[fibula_r_midshaft_anter][0], 
                               lms_list[fibula_r_midshaft_anter][1], 
                               tibia_r_mean[2])
lms.SetNthControlPointPosition(tibia_r_midshaft_anter, 
                               lms_list[tibia_r_midshaft_anter][0], 
                               lms_list[tibia_r_midshaft_anter][1], 
                               tibia_r_mean[2])
lms.SetNthControlPointPosition(tibia_r_midshaft_poster, 
                               lms_list[tibia_r_midshaft_poster][0], 
                               lms_list[tibia_r_midshaft_poster][1], 
                               tibia_r_mean[2])

In [ ]:
### reorient the red slice:
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, tibia_r_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, adjust landmarks position to fit on this slice
</div>

In [ ]:
## Left femur
lms = getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_center_coords = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
femur_l_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
femur_l_mean = np.mean((knee_l_center_coords, femur_l_center_coords), axis = 0)

femur_l_midshaft_anter = lms.GetControlPointIndexByLabel('femur_l_midshaft_anter')
lms.SetNthControlPointPosition(femur_l_midshaft_anter, 
                               lms_list[femur_l_midshaft_anter][0], 
                               lms_list[femur_l_midshaft_anter][1], 
                               femur_l_mean[2])

femur_l_midshaft_poster = lms.GetControlPointIndexByLabel('femur_l_midshaft_poster')
lms.SetNthControlPointPosition(femur_l_midshaft_poster, 
                               lms_list[femur_l_midshaft_poster][0], 
                               lms_list[femur_l_midshaft_poster][1], 
                               femur_l_mean[2])

In [ ]:
### reorient the red slice:
femur_l_normal = (femur_l_center_coords-knee_l_center_coords)/np.linalg.norm(femur_l_center_coords - knee_l_center_coords)
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_l_normal, femur_l_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust landmarks position to fit on the above slice.
</div>

In [ ]:
## Right femur
lms = getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center_coords = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
femur_r_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
femur_r_mean = np.mean((knee_r_center_coords, femur_r_center_coords), axis = 0)

femur_r_midshaft_anter = lms.GetControlPointIndexByLabel('femur_r_midshaft_anter')
lms.SetNthControlPointPosition(femur_r_midshaft_anter, 
                               lms_list[femur_r_midshaft_anter][0], 
                               lms_list[femur_r_midshaft_anter][1], 
                               femur_r_mean[2])

femur_r_midshaft_poster = lms.GetControlPointIndexByLabel('femur_r_midshaft_poster')
lms.SetNthControlPointPosition(femur_r_midshaft_poster, 
                               lms_list[femur_r_midshaft_poster][0], 
                               lms_list[femur_r_midshaft_poster][1], f
                               emur_r_mean[2])

In [ ]:
### reorient the red slice:
femur_r_normal = (femur_r_center_coords-knee_r_center_coords)/np.linalg.norm(femur_r_center_coords - knee_r_center_coords)
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_r_normal, femur_r_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust landmarks position to fit on the above slice.
</div>

In [ ]:
### return red slice to normal orientation
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), femur_r_mean)

<div class="alert alert-block alert-info">
<b>Patellas -- special attention!</b>
Adjust the position of patella markers manually. Note that lateral and medial points shoul be approximately on one line. Also, _patella__[side]_ and _patella__[side]_in_femur__[side]_ -- the same position at the lowest point of the left patella
</div>

In [ ]:
# Update location of joints in parent frames that were previousely omitted -- these should have been projected into the correct location anyway but just to make sure
lms = getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

pelvis_origin_in_ground_index = lms.GetControlPointIndexByLabel('pelvis_origin_in_ground')
pelvis_origin_coords = lms_list[lms.GetControlPointIndexByLabel('pelvis_origin')]
lms.SetNthControlPointPosition(pelvis_origin_in_ground_index, 
                               pelvis_origin_coords[0], 
                               pelvis_origin_coords[1], 
                               pelvis_origin_coords[2])

femur_l_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
femur_l_center_in_pelvis_index = lms.GetControlPointIndexByLabel('femur_l_center_in_pelvis')
lms.SetNthControlPointPosition(femur_l_center_in_pelvis_index, 
                               femur_l_center_coords[0], 
                               femur_l_center_coords[1], 
                               femur_l_center_coords[2])

femur_r_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
femur_r_center_in_pelvis_index = lms.GetControlPointIndexByLabel('femur_r_center_in_pelvis')
lms.SetNthControlPointPosition(femur_r_center_in_pelvis_index, 
                               femur_r_center_coords[0], 
                               femur_r_center_coords[1], 
                               femur_r_center_coords[2])